In [1]:
import torch
from torch import nn
from torch.nn import MSELoss
from torch.optim import Adam
from LSTM_model.data_loader import StockData
from LSTM_model.model import StockModel
import pandas as pd
import numpy as np

In [2]:
if torch.cuda.is_available():
    print("GPUs are available")
    device = "cuda"
else: 
    print("GPU is not available")
    device = "cpu"

GPUs are available


In [3]:
data = pd.read_csv("stock_data.csv")
company_name = "MSFT"
data = data[data["Stock"] == company_name]

In [4]:
from sklearn.preprocessing import MinMaxScaler

price = data[['Close']]
scaler = MinMaxScaler(feature_range=(-1, 1))
price = scaler.fit_transform(price['Close'].values.reshape(-1,1))

In [5]:
def split_data(stock, lookback):
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(stock) - lookback): 
        data.append(stock[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

In [6]:
lookback = 20
x_train, y_train, x_test, y_test = split_data(price, lookback)

In [7]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)

y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)

In [8]:
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  torch.Size([6370, 19, 1])
y_train.shape =  (6370, 1)
x_test.shape =  torch.Size([1593, 19, 1])
y_test.shape =  (1593, 1)


In [9]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 500

In [10]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().cuda()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().cuda()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [11]:
lstm_model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).cuda()
criterion = torch.nn.MSELoss(reduction='mean').cuda()
optimiser = torch.optim.Adam(lstm_model.parameters(), lr=0.01)

In [12]:
import math, time
from sklearn.metrics import mean_squared_error

def validate(model, x_test, y_test):
    # make predictions
    y_test_pred = model(x_test.cuda()).cpu()

    # invert predictions
    y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
    y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())

    # calculate root mean squared error
    testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
    
    return testScore

In [13]:
import time

hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []

save_losses = {}
save_losses["min_test_loss"] = 10e10
for epoch in range(num_epochs):
    y_train_pred = lstm_model(x_train.cuda())

    loss = criterion(y_train_pred.cuda(), y_train_lstm.cuda())
    test_loss = validate(lstm_model, x_test, y_test)
    print(f"Epoch {epoch} - MSE: {loss.item()} - Test MSE: {test_loss}")
    hist[epoch] = loss.item()
    if save_losses["min_test_loss"] > test_loss:
        save_losses["min_test_loss"] = test_loss
        save_losses["train_loss"] = loss.item()
        save_losses["epoch"] = epoch
        
        torch.save(lstm_model.state_dict(), f"checkpoints/{company_name}/{epoch}_lstm32_{test_loss}.pth")

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

Epoch 0 - MSE: 0.3771309554576874 - Test MSE: 15.820912893699878
Epoch 1 - MSE: 0.21688629686832428 - Test MSE: 18.152501361081413
Epoch 2 - MSE: 0.07675184309482574 - Test MSE: 24.332812299332176
Epoch 3 - MSE: 0.16591386497020721 - Test MSE: 39.644614193737745
Epoch 4 - MSE: 0.07939515262842178 - Test MSE: 33.88804054518626
Epoch 5 - MSE: 0.055317457765340805 - Test MSE: 28.39656222315916
Epoch 6 - MSE: 0.06929023563861847 - Test MSE: 25.481997848588495
Epoch 7 - MSE: 0.08094321191310883 - Test MSE: 24.25248784153713
Epoch 8 - MSE: 0.08282765746116638 - Test MSE: 23.992144252647442
Epoch 9 - MSE: 0.07733430713415146 - Test MSE: 24.333587366459014
Epoch 10 - MSE: 0.06830987334251404 - Test MSE: 25.092385450912943
Epoch 11 - MSE: 0.059339188039302826 - Test MSE: 26.152811387249198
Epoch 12 - MSE: 0.05334130674600601 - Test MSE: 27.40156722215846
Epoch 13 - MSE: 0.051979560405015945 - Test MSE: 28.682327369310748
Epoch 14 - MSE: 0.0547497533261776 - Test MSE: 29.77376242114754
Epoch 15 

In [14]:
print(save_losses)

{'min_test_loss': 5.345173607086357, 'train_loss': 0.0002036334335571155, 'epoch': 326}
